In [6]:
import pandas as pd
STRchive = pd.read_json('/Users/quinlan/Documents/Git/STRchive_manuscript/data/STRchive-database.json')
gnomAD = pd.read_json('/Users/quinlan/Documents/Git/STRchive_manuscript/data/gnomADcatalog1.json')
# Rename 'Gene' column to 'gene' in gnomAD DataFrame
gnomAD.rename(columns={'Gene': 'gene'}, inplace=True)



In [7]:
# Filter gnomAD and STRchive DataFrames to keep only rows with gene values present in both
common_genes = set(gnomAD['gene']).intersection(set(STRchive['gene']))
gnomAD_filtered = gnomAD[gnomAD['gene'].isin(common_genes)]
STRchive_filtered = STRchive[STRchive['gene'].isin(common_genes)]


In [10]:
gene_count = {}

# Iterate through the 'gene' column
for index, row in STRchive_filtered.iterrows():
    gene = row['gene']
    # Check if the gene has occurred before
    if gene in gene_count:
        # Increment the count for this gene
        gene_count[gene] += 1
        # Rename the gene with a numeric suffix
        STRchive_filtered.loc[index, 'gene'] = f"{gene}_{gene_count[gene]}"
    else:
        # Initialize count for this gene
        gene_count[gene] = 1

In [11]:
gene_count = {}

# Iterate through the 'gene' column
for index, row in gnomAD_filtered.iterrows():
    gene = row['gene']
    # Check if the gene has occurred before
    if gene in gene_count:
        # Increment the count for this gene
        gene_count[gene] += 1
        # Rename the gene with a numeric suffix
        gnomAD_filtered.loc[index, 'gene'] = f"{gene}_{gene_count[gene]}"
    else:
        # Initialize count for this gene
        gene_count[gene] = 1

In [12]:
# Create ReferenceRegion column
STRchive_filtered['ReferenceRegion'] = STRchive_filtered['chrom'] + ':' + \
                                       STRchive_filtered['start_hg38'].astype(str) + '-' + \
                                       STRchive_filtered['stop_hg38'].astype(str)


/var/folders/kx/jkrbpc3n5lggz22pmw8lghth0000gp/T/ipykernel_83994/2032255851.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STRchive_filtered['ReferenceRegion'] = STRchive_filtered['chrom'] + ':' + \


In [13]:
# Select only 'gene' and 'ReferenceRegion' columns from gnomAD_filtered DataFrame
gnomAD_filtered = gnomAD_filtered[['gene', 'MainReferenceRegion']]
gnomAD_filtered.rename(columns={'MainReferenceRegion': 'ReferenceRegion'}, inplace=True)
# Select only 'gene' and 'ReferenceRegion' columns from STRchive_filtered DataFrame
STRchive_filtered = STRchive_filtered[['gene', 'ReferenceRegion']]


In [14]:
# same ReferenceRegion
merged_df = gnomAD_filtered[gnomAD_filtered['ReferenceRegion'].isin(STRchive_filtered['ReferenceRegion'])]
# 27 loci are the same


In [15]:
gnomAD_filtered[['chrom', 'start', 'stop']] = gnomAD_filtered['ReferenceRegion'].str.extract(r'(chr\d+|chrX):(\d+)-(\d+)')
STRchive_filtered[['chrom', 'start', 'stop']] = STRchive_filtered['ReferenceRegion'].str.extract(r'(chr\d+|chrX):(\d+)-(\d+)')



/var/folders/kx/jkrbpc3n5lggz22pmw8lghth0000gp/T/ipykernel_83994/273072076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STRchive_filtered[['chrom', 'start', 'stop']] = STRchive_filtered['ReferenceRegion'].str.extract(r'(chr\d+|chrX):(\d+)-(\d+)')
/var/folders/kx/jkrbpc3n5lggz22pmw8lghth0000gp/T/ipykernel_83994/273072076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STRchive_filtered[['chrom', 'start', 'stop']] = STRchive_filtered['ReferenceRegion'].str.extract(r'(chr\d+|chrX):(\d+)-(\d+)')
/var

In [16]:
merged_df = pd.merge(gnomAD_filtered, STRchive_filtered, on='gene', suffixes=('_gnomAD', '_STRchive'), how='inner')


In [17]:
# Convert start and stop columns to numeric type
merged_df['start_gnomAD'] = pd.to_numeric(merged_df['start_gnomAD'])
merged_df['stop_gnomAD'] = pd.to_numeric(merged_df['stop_gnomAD'])
merged_df['start_STRchive'] = pd.to_numeric(merged_df['start_STRchive'])
merged_df['stop_STRchive'] = pd.to_numeric(merged_df['stop_STRchive'])

# Calculate the differences
merged_df['start_difference'] = merged_df['start_STRchive'] - merged_df['start_gnomAD']
merged_df['stop_difference'] = merged_df['stop_STRchive'] - merged_df['stop_gnomAD']

# Print the results with the gene name
for index, row in merged_df.iterrows():
    print(f"Gene: {row['gene']}, Start Difference: {row['start_difference']}, Stop Difference: {row['stop_difference']}")


Gene: ABCD3, Start Difference: 1, Stop Difference: 2
Gene: AFF2, Start Difference: -26, Stop Difference: 62
Gene: AFF3, Start Difference: 1, Stop Difference: 2
Gene: AR, Start Difference: 1, Stop Difference: 34
Gene: ARX, Start Difference: 0, Stop Difference: 0
Gene: ARX_2, Start Difference: 1, Stop Difference: 0
Gene: ATN1, Start Difference: 1, Stop Difference: 2
Gene: ATXN1, Start Difference: 1, Stop Difference: 1
Gene: ATXN10, Start Difference: 1, Stop Difference: 0
Gene: ATXN2, Start Difference: 1, Stop Difference: 1
Gene: ATXN3, Start Difference: 2, Stop Difference: 13
Gene: ATXN7, Start Difference: 1, Stop Difference: 2
Gene: BEAN1, Start Difference: -1, Stop Difference: 13
Gene: C9orf72, Start Difference: -44, Stop Difference: 0
Gene: CACNA1A, Start Difference: 1, Stop Difference: 1
Gene: CBL, Start Difference: 0, Stop Difference: 0
Gene: CNBP, Start Difference: 1, Stop Difference: 3
Gene: COMP, Start Difference: 0, Stop Difference: 0
Gene: CSTB, Start Difference: 0, Stop Differ